In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

import keras_tuner as kt

In [2]:
df = pd.read_csv("D:/Intern/DataSets/Pima-Indians-Diabetes.csv")

In [3]:
df.shape

(768, 9)

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(8,)))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4756 - loss: 0.7628 - val_accuracy: 0.5584 - val_loss: 0.7173
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5733 - loss: 0.6818 - val_accuracy: 0.6364 - val_loss: 0.6595
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6580 - loss: 0.6233 - val_accuracy: 0.6494 - val_loss: 0.6183
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6873 - loss: 0.5839 - val_accuracy: 0.6429 - val_loss: 0.5905
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7068 - loss: 0.5553 - val_accuracy: 0.6688 - val_loss: 0.5698
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7345 - loss: 0.5333 - val_accuracy: 0.6883 - val_loss: 0.5553
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7524 - loss: 0.5167 - val_accuracy: 0.7143 - val_loss: 0.5438
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7671 - loss: 0.5044 - val_accuracy: 0.7078 - val_loss

In [11]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}, Accuracy: {acc:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7468 - loss: 0.5243 
Loss: 0.5243, Accuracy: 0.7468


### Tunning

In [12]:
def build_model(hp):
    model = Sequential()
    
    units = hp.Int('units', min_value=8, max_value=128, step=8)
    activation = hp.Choice('activation', values = ['relu', 'tanh'])
    model.add(Dense(units=units, activation=activation, input_shape=(8,)))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [13]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='my_dir', project_name='Keras_Tuner_Pima-Diabetes-1')

In [14]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.4740259647369385

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 08s


In [15]:
tuner.get_best_hyperparameters()[0].values

{'units': 96, 'activation': 'relu', 'optimizer': 'adam'}

In [16]:
model = tuner.get_best_models(num_models=1)[0]
model.fit(X_train, y_train, epochs=20, initial_epoch=6, batch_size=32, validation_data=(X_test, y_test))

Epoch 7/20


C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7671 - loss: 0.4817 - val_accuracy: 0.7468 - val_loss: 0.5212
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7687 - loss: 0.4640 - val_accuracy: 0.7597 - val_loss: 0.5183
Epoch 9/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7687 - loss: 0.4563 - val_accuracy: 0.7468 - val_loss: 0.5153
Epoch 10/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7785 - loss: 0.4487 - val_accuracy: 0.7597 - val_loss: 0.5130
Epoch 11/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7785 - loss: 0.4447 - val_accuracy: 0.7532 - val_loss: 0.5135
Epoch 12/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7899 - loss: 0.4411 - val_accuracy: 0.7532 - val_loss: 0.5127
Epoch 13/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7850 - loss: 0.4381 - val_accuracy: 0.7468 - val_loss: 0.5130
Epoch 14/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7866 - loss: 0.4364 - val_accuracy: 0.7403 - val_loss: 0.51

In [17]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}, Accuracy: {acc:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7597 - loss: 0.5195 
Loss: 0.5195, Accuracy: 0.7597


### Layer tunnung

In [21]:
def build_model(hp):
    model = Sequential()
    
    counter = 0
    
    for i in range(hp.Int('num_layers', min_value=1, max_value=10))  :
        if counter == 0:
            # first layer
            model.add(
                Dense(units=hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation' + str(i), values = ['relu', 'tanh', 'sigmoid']), 
                    input_dim=8))
            model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3,0.4,0.5,0.6])))
        else:
            # hidden layers
            model.add(
                Dense(units=hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                      activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])))
            
        counter += 1
    
	# last layer
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'sgd', 'RMSprop', 'adadelta']), loss='binary_crossentropy', metrics=['accuracy'])
        
    return model

In [22]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=3, directory='my_dir', project_name='Keras_Tuner_Pima-Diabetes-2')

Reloading Tuner from my_dir\Keras_Tuner_Pima-Diabetes-2\tuner0.json


In [23]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 01m 38s


In [24]:
parameters = tuner.get_best_hyperparameters()[0].values
parameters

{'num_layers': 10,
 'units0': 16,
 'activation0': 'relu',
 'dropout0': 0.3,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'units2': 8,
 'activation2': 'relu',
 'units3': 8,
 'activation3': 'relu',
 'units4': 8,
 'activation4': 'relu',
 'units5': 8,
 'activation5': 'relu',
 'units6': 8,
 'activation6': 'relu',
 'units7': 8,
 'activation7': 'relu',
 'units8': 8,
 'activation8': 'relu',
 'units9': 8,
 'activation9': 'relu'}

In [25]:
best_model = tuner.get_best_models(num_models=1)[0]

C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [26]:
model.fit(X_train, y_train, epochs=20, initial_epoch=6, batch_size=32, validation_data=(X_test, y_test))

Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7932 - loss: 0.4248 - val_accuracy: 0.7662 - val_loss: 0.5194
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7899 - loss: 0.4235 - val_accuracy: 0.7662 - val_loss: 0.5185
Epoch 9/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7899 - loss: 0.4214 - val_accuracy: 0.7662 - val_loss: 0.5178
Epoch 10/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7899 - loss: 0.4203 - val_accuracy: 0.7662 - val_loss: 0.5210
Epoch 11/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7915 - loss: 0.4190 - val_accuracy: 0.7597 - val_loss: 0.5225
Epoch 12/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7948 - loss: 0.4178 - val_accuracy: 0.7662 - val_loss: 0.5198
Epoch 13/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7915 - loss: 0.4165 - val_accuracy: 0.7662 - val_loss: 0.5226
Epoch 14/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7915 - loss: 0.4157 - val_accuracy: 0.7662 - val_

In [27]:
loass, acc = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}, Accuracy: {acc:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7532 - loss: 0.5347 
Loss: 0.5195, Accuracy: 0.7532
